### Semaphores in C

[Pthreads](https://computing.llnl.gov/tutorials/pthreads/) is a library commonly available for Unix and other operating systems. It is standardized by POSIX. In C, using Pthreads requires:
```C
# include <pthread.h>
```
For each thread, the following variables have to be declared and initialized:
```C
pthread_attr_t tattr;       /* thread attributes */
pthread_t tid;              /* thread descriptor */

pthread_attr_init(&tattr);  /* default values */
pthread_attr_setscope(&tattr, PTHREAD_SCOPE_SYSTEM);
```
The default values include a default stack size. Threads can compete for scheduling locally (default), i.e. with threads of the same process, globally (system scope), i.e. with all other threads. The fork operation takes a previously set attribute descriptor, the function to be started, and its parameters and initializes a thread descriptor (first argument):
```C
pthread_create(&tid, &tattr, start_func, arg);
```
A thread terminates if the function body completes or by calling:
```C
pthread_exit(value);
```
A parent thread can wait for a child to terminate; it needs to specify the descriptor and the location for the return value of the child:
```C
pthread_join(tid, value_ptr);
```

Threads can synchronize with several mechanisms, including semaphores. A semaphore is declared by:
```C
sem_t mutex;
```
Semaphores must be initialized explicitly. If the `SHARED` parameter is non-zero, it can be shared between processes as well, otherwise only between threads:
```C
sem_init(&mutex, SHARED, 1);
```
The pattern for mutual exclusion with semaphores is:
```C
sem_wait(&mutex);			/* P(mutex) */
critical section
sem_post(&mutex);			/* V(mutex) */ 
```
The value of a semaphore can be obtained by calling `sem_getvalue(&mutex, &value);`. If no thread is blocked on `mutex`, it can be destroyed by `sem_destroy(&mutex);`.

Here is the producer/consumer problem with semaphores in C. The shared variable `data` is declared as `volatile`. This tells the compiler that the variable can change at any time and has to be read from memory. Without that, in the function `Consumer`, the compiler may assume that `data` does not change in the function and keep it in a register.

In [ ]:
%%writefile pc.c

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <semaphore.h>
#define SHARED 1

void *Producer(void *);           /* the two threads */
void *Consumer(void *);

sem_t empty, full;                /* the global semaphores */
volatile int data;                /* shared buffer         */
int numIters;

/* main program: read command line and create threads */
int main(int argc, char *argv[]) {
    pthread_t pid, cid;           /* thread and attributes */
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    pthread_attr_setscope(&attr, PTHREAD_SCOPE_SYSTEM);
    sem_init(&empty, SHARED, 1);  /* sem empty = 1 */
    sem_init(&full, SHARED, 0);   /* sem full = 0  */
    numIters = atoi(argv[1]);
    pthread_create(&pid, &attr, Producer, NULL);
    pthread_create(&cid, &attr, Consumer, NULL);
    pthread_join(pid, NULL);
    pthread_join(cid, NULL);
}

/* deposit 1, ..., numIters into the data buffer */
void *Producer(void *arg) {
    printf("Producer created\n");
    for (int produced = 0; produced < numIters; produced++) {
        sem_wait(&empty);
        data = produced;
        sem_post(&full);
    }
}

/* fetch numIters items from the buffer and sum them */
void *Consumer(void *arg) {
    printf("Consumer created\n");
    int sum = 0;
    for (int consumed = 0; consumed < numIters; consumed++) {
        sem_wait(&full);
        sum += data;
        sem_post(&empty);
    }
    printf("For %d iterations, the sum is %d\n", numIters, sum);
}

Compile and run the code. The numbers from 0 to the specified command line parameter are produced and consumed. The consumer prints the sum of the consumed numbers.

In [ ]:
!gcc pc.c -lpthread -o pc

In [ ]:
!./pc 5